In [1]:
# Use the conda env machine learning to work things

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import scipy.sparse as sparse
import re


Reading in Insulator Sequences and randomly sampled sequenes

In [2]:
random_sequences = pd.read_csv('1000_8kb_random_samples_upper.tab',sep='\t',names=['Region','Sequence'])
insulator_sequences = pd.read_csv('insulator_library.tab',sep='\t',names=['Region','Sequence'])
combined_sequences=insulator_sequences.append(random_sequences)

combined_sequences.head()


#random_sample=random_sequences.sample(n=5000,random_state=1)
#insulator_sample=insulator_sequences.sample(n=990,random_state=1)

,Region,Sequence
0,Insulator1,CAGGGTTGCCCAGTGCCCCTTGTCACCCCCCGAGCAACACAAGCCA...
1,Insulator2,TTCCTCGTGTGCAGAAATCGGGGGTCAGGGGATGGTTCACATGTGG...
2,Insulator3,AACAGGAAATCCCATTCCCCCATGCCAGTGACTACACCAGGGAAGG...
3,Insulator4,TTAAAATTGTAATGTGTCCTGATTTAGAGGAAACTAATTATTATGT...
4,Insulator5,GTACACACTGGCTCAGTGGATTCATATCCAAAAAGCTGAGCATTGA...


In [ ]:
insulator_dataset = combined_sequences.copy() 
ins_data = insulator_dataset['Sequence']

motif_dict={} 

for entry in ins_data:
    for i in range(len(entry)-5):
        sequence = entry[i : i+5]
        if sequence not in motif_dict:
            motif_dict[sequence]=1
        else: 
            motif_dict[sequence]+=1
    
motif_list = list(motif_dict)


def motif_counter(sequence):
    individual_dict = dict.fromkeys(motif_list, 0)
    for i in range(len(sequence)-5):
        motif = sequence[i : i+5]
        individual_dict[motif] += 1
    return individual_dict

motif_df = pd.DataFrame()

for entry in ins_data:
    dictionary = motif_counter(entry)
    single_motif_df = pd.DataFrame(dictionary, index = [0])
    motif_df = motif_df.append(single_motif_df)

motif_df.index = range(len(motif_df))
insulator_dataset.index = range(len(insulator_dataset))

insulator_dataset = insulator_dataset.join(motif_df)
motif_df.to_csv('Insulator_motif_dictionary_ext.csv')

In [16]:
from sklearn.ensemble import RandomForestClassifier

ValueError: Found array with 0 feature(s) (shape=(9352, 0)) while a minimum of 1 is required.

In [104]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred)
print(cm)

[[2350   53]
 [ 522  193]]


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}



# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_x, train_y)

print(rf_random.best_params_)



forest_model = RandomForestClassifier(random_state = 1)
forest_model.fit(train_x, train_y)
preds = forest_model.predict(test_x)
print(mean_absolute_error(test_y, preds))

#Permutation importance
#perm = PermutationImportance(forest_model, random_state = 1).fit(test_x, test_y)
#eli5.show_weights(perm,feature_names = test_x.columns.tolist())
from xgboost import XGBRegressor

XGB_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05)
XGB_model.fit(train_x, train_y, early_stopping_rounds=5, eval_set=[(test_x, test_y)], verbose = False)

XGB_preds = XGB_model.predict(test_x)
print(mean_absolute_error(test_y, XGB_preds))




#other stuff I tried:

#new_splitprime = ANN_dataset['seq_3utr'].str.split("", expand = True) 
#ANN_dataset = ANN_dataset.drop(['seq_3utr', 'index'], axis = 1)
#ANN_dataset = ANN_dataset.join(new_splitprime)

#from numpy import array
#from numpy import argmax
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder
#import re


#onehot = []

#for index, row in ANN_dataset.iterrows():
#    seq_array = array(list(row['seq_3utr']))
#    label_encoder = LabelEncoder()
#    integer_encoded_seq = label_encoder.fit_transform(seq_array)
#    onehot_encoder = OneHotEncoder(sparse=False)
#    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
#    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
#    onehot.append(onehot_encoded_seq)

#ANN_dataset['onehotencoded'] = onehot

#ANN_y = ANN_dataset['Halflife']
#ANN_x = ANN_dataset['onehotencoded']

#for row in ANN_x:
#    np.split(row, 4)

#for row in ANN_x:
#    for i in ANN_x:
#        RNA_seq = ANN_x[i : i+5]

In [ ]:
prediction=clf.predict(test_x[1])

In [ ]:
print(mean_absolute_error(test_y, preds))